In [ ]:
n_qubits = 3
depth = 1
ansatz_id = 9
subgroup_size = 1

import json
from ansatz import Ansatz
from utils import find_original_param, get_subgroup_unitaries, pennylane_to_qiskit

from twirlator.symmetry_groups import create_symmetric_group
from twirlator.generators import get_ansatz_generators
from twirlator.twirling import apply_twirling_to_generators

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import HamiltonianGate #PauliEvolutionGate, UnitaryGate
from qiskit.quantum_info import SparsePauliOp
    
S = create_symmetric_group(n_qubits)

with open(f"groups/subgroups_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

subgroup_unitaries = get_subgroup_unitaries(subgroups, S)

super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
circuit, params = super_ansatz.get_QNode()
qiskit_circuit = pennylane_to_qiskit(circuit, n_qubits, params=params)
qiskit_circuit = qiskit_circuit.remove_final_measurements(inplace=False)
print(qiskit_circuit.draw("latex_source"))
params = qiskit_circuit.parameters
ansatz_generators = get_ansatz_generators(super_ansatz.get_ansatz())

for elem in subgroup_unitaries[str(subgroup_size)]:
    unitaries = elem["unitaries"]
    print(unitaries)
    twirled_generators = apply_twirling_to_generators(unitaries, ansatz_generators, n_qubits)
    twirled_circuit = QuantumCircuit(n_qubits)
    for i, (gen_matrix, op_wires, op_name, theta, is_parametric) in enumerate(ansatz_generators):
        twirled_elem = twirled_generators[i]
        if twirled_elem["gate_name"] == op_name and twirled_elem["wires"] == op_wires:
            H = twirled_elem['averaged']
            if is_parametric:
                param = find_original_param(qiskit_circuit, ansatz_generators, i, op_name, op_wires, theta)
            else:
                param = theta
                if op_name == "Hadamard":
                    print(theta)
            pauli_op = SparsePauliOp.from_operator(H)
            evo_gate = HamiltonianGate(pauli_op, time=param)
            twirled_circuit.append(evo_gate, range(n_qubits))
        else:
            raise ValueError(f"Twirled generator for {op_name} on wires {op_wires} not found when {twirled_elem['gate_name']} and {twirled_elem['wires']}")

    twirled_circuit.remove_final_measurements(inplace=True)
    twirled_circuit = twirled_circuit.assign_parameters({param : 1.0 for param in twirled_circuit.parameters})
    transpiled_circuit = transpile(
            twirled_circuit,
            basis_gates=['rz', 'sx', 'cx', 'h', 'x', 'y', 'z', 'rx', 'ry', 'cz', 'rxx', 'rzz'],
            optimization_level=3
        )
        # Print more decimals in the figure labels
    fig = transpiled_circuit.draw("mpl")
    fig.savefig(f"twirled_circuit_n{n_qubits}_d{depth}_a{ansatz_id}_k{subgroup_size}.png")
    print(transpiled_circuit.draw("latex_source"))
    break


\documentclass[border=2px]{standalone}

\usepackage[braket, qm]{qcircuit}
\usepackage{graphicx}

\begin{document}
\scalebox{1.0}{
\Qcircuit @C=1.0em @R=0.2em @!R { \\
	 	\nghost{{q}_{0} :  } & \lstick{{q}_{0} :  } & \gate{\mathrm{H}} & \qw & \control\qw & \gate{\mathrm{R_X}\,(\mathrm{x0})} & \qw & \qw\\
	 	\nghost{{q}_{1} :  } & \lstick{{q}_{1} :  } & \gate{\mathrm{H}} & \control\qw & \ctrl{-1} & \gate{\mathrm{R_X}\,(\mathrm{x1})} & \qw & \qw\\
	 	\nghost{{q}_{2} :  } & \lstick{{q}_{2} :  } & \gate{\mathrm{H}} & \ctrl{-1} & \gate{\mathrm{R_X}\,(\mathrm{x2})} & \qw & \qw & \qw\\
\\ }}
\end{document}
{0: array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0

In [17]:
import numpy as np
import pennylane as qml
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import HGate

pauli_op = SparsePauliOp.from_list([
        ("X", 1/np.sqrt(2)),
        ("Z", 1/np.sqrt(2)),
    ])
theta = np.pi / 2

param = theta


gen_observable = pauli_op.to_matrix()
full_n_qubits = 3
twirled_circuit = QuantumCircuit(full_n_qubits)
op = qml.Hermitian(gen_observable, wires=[0])
assert op.is_hermitian, "Generator observable must be Hermitian"
G_full = qml.matrix(op, wire_order=range(full_n_qubits))
print(G_full)  # Should print a 2x2 Hermitian matrix
pauli_op = SparsePauliOp.from_operator(G_full)
evo_gate = HamiltonianGate(pauli_op, time=theta)
#print(evo_gate.operator.to_matrix())  # Should print the same Hermitian matrix as G_full
twirled_circuit.append(evo_gate, range(full_n_qubits))
twirled_circuit.remove_final_measurements(inplace=True)
transpiled_circuit = transpile(
        twirled_circuit,
        basis_gates=['rz', 'sx', 'cx', 'h', 'x', 'y', 'z', 'rx', 'ry', 'cz', 'rxx', 'rzz'],
        optimization_level=3
    )
    # Print more decimals in the figure labels
fig = transpiled_circuit.draw("mpl")
fig.savefig(f"twirled_circuit_hadamard_test.png")


[[ 0.70710678+0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.70710678+0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.70710678+0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.70710678+0.j]
 [ 0.70710678+0.j  0.        +0.j  0.        +0.j  0.        +0.j
  -0.70710678+0.j -0.        +0.j -0.        +0.j -0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.        +0.j
  -0.        +0.j -0.70710678+0.j -0.        +0.j -0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j  0.        +0.j
  -0.        +0.j -0.        +0.j -0.70710678+0.j -0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.70710678+0.j
  -

In [18]:
#Print gates and their parameters
for qarg in transpiled_circuit.data:
    print(qarg)

CircuitInstruction(operation=Instruction(name='rx', num_qubits=1, num_clbits=0, params=[3.141592653589793]), qubits=(Qubit(QuantumRegister(3, 'q'), 1),), clbits=())
CircuitInstruction(operation=Instruction(name='rx', num_qubits=1, num_clbits=0, params=[-1.677382718227164]), qubits=(Qubit(QuantumRegister(3, 'q'), 2),), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())
CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-3.1415926535897922]), qubits=(Qubit(QuantumRegister(3, 'q'), 2),), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())
CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-3.141592653589795]), qubits=(Qubit(QuantumRegiste

In [ ]:
import numpy as np
import pennylane as qml
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import PauliEvolutionGate, HamiltonianGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import HGate, CZGate
from qiskit.quantum_info import Statevector

# Verify with PennyLane
gen_observable, theta = SparsePauliOp.from_list([("X", 1/np.sqrt(2)), ("Z", 1/np.sqrt(2))]), np.pi / 2
op0 = qml.Hermitian(gen_observable.to_matrix(), wires=[0])
G_full0 = qml.matrix(op0, wire_order=range(2))
op1 = qml.Hermitian(gen_observable.to_matrix(), wires=[1])
G_full1 = qml.matrix(op1, wire_order=range(2))

gen_observable2, theta2 = SparsePauliOp.from_list([("II",  1/4),("IZ", -1/4),("ZI", -1/4),("ZZ",  1/4)]), np.pi
op2 = qml.Hermitian(gen_observable2.to_matrix(), wires=[0, 1])
G_full2 = qml.matrix(op2, wire_order=range(2))

# Flip between PauliEvolutionGate and HamiltonianGate to see the difference
if True:
    evo_gate1 = PauliEvolutionGate(SparsePauliOp.from_operator(G_full0), time=theta)
    evo_gate2 = PauliEvolutionGate(SparsePauliOp.from_operator(G_full1), time=theta)
    evo_gate3 = PauliEvolutionGate(SparsePauliOp.from_operator(G_full2), time=theta2)
else:
    evo_gate1 = HamiltonianGate(SparsePauliOp.from_operator(G_full0), time=theta)
    evo_gate2 = HamiltonianGate(SparsePauliOp.from_operator(G_full1), time=theta)
    evo_gate3 = HamiltonianGate(SparsePauliOp.from_operator(G_full2), time=theta2)

qc = QuantumCircuit(2)
qc.append(evo_gate1, range(2))
#qc.append(evo_gate2, range(2))
#qc.append(evo_gate3, [0, 1])

# Confirm that the circuit is the same as applying Hadamard to both qubits and CZ between them
qc_h_cz = QuantumCircuit(2)
qc_h_cz.append(HGate(), [0])
#qc_h_cz.append(HGate(), [1])
#qc_h_cz.append(CZGate(), [0, 1])

Statevector.from_instruction(qc).equiv(Statevector.from_instruction(qc_h_cz), atol=1e-1)

False

In [ ]:
import numpy as np
from qiskit.circuit.library import PauliEvolutionGate, HGate
from qiskit.quantum_info import SparsePauliOp

pauli_op = SparsePauliOp.from_list([("X", 1/np.sqrt(2)), ("Z", 1/np.sqrt(2))])
theta = np.pi / 2

U_evo = PauliEvolutionGate(pauli_op, time=theta).operator.to_matrix()
U_H = HGate().to_matrix()

U_total = U_H @ U_evo
print(np.round(U_total, 6))


[[ 1.+0.j -0.+0.j]
 [-0.+0.j  1.+0.j]]
